In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from uproot3_methods.classes.TLorentzVector import TLorentzVector
mag2 = TLorentzVector.mag2

In [ ]:
file = up.open("../build/evtOutput.root")

In [ ]:
class rootDF:
    def __init__(self, file_name):
        file = up.open(file_name)
        ntp = file["ntp"]
        self.Px = ntp["px"].array()
        self.Py = ntp["py"].array()
        self.Pz = ntp["pz"].array()
        self.E = ntp["E"].array()
        self.nTrk = ntp["nTrk"].array()
        file.close()
    def sign(self, x):
        return -1 if x<0 else 1
    def get_p_from_list(self, iEv, part_list):
        p = TLorentzVector(0, 0, 0, 0)
        for i in part_list:
            if( abs(max(part_list)) >= self.nTrk[iEv]):
                continue
            abs_i = abs(i)
            p += self.sign(i)*TLorentzVector(
                self.Px[iEv][abs_i], self.Py[iEv][abs_i], self.Pz[iEv][abs_i], self.E[iEv][abs_i]
            )
        return p
    def size(self):
        return len(self.Px)
    def apply(self, func, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        try:
            return func.fget(p)
        except ValueError:
            return 0
    def m2(self, iEv, part_list):
        return self.apply(TLorentzVector.mag2, iEv, part_list)
    def m(self, iEv, part_list):
        return self.apply(TLorentzVector.mag, iEv, part_list)
    def pt2(self, iEv, part_list):
        return self.apply(TLorentzVector.pt2, iEv, part_list)
    def pt(self, iEv, part_list):
        return self.apply(TLorentzVector.pt, iEv, part_list)
        

In [ ]:
df = rootDF('../build/evtOutput.root')

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows = 1, figsize = (15,7))
mm_list = [ [df.m2(iEv, [2,3]), df.m(iEv, [0,-1])] for iEv in range(df.size() )]
mm_list = np.array(mm_list)
mm_list = np.array([mm for mm in mm_list if sum(mm**2)> 1e-5])
axes[0].hist(mm_list[:,0], bins =30)
axes[1].hist(mm_list[:,1]**2, bins = 30)
plt.suptitle('M2')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows = 1, figsize = (15,7))
mm_list = [ [df.pt2(iEv, [2,3]), df.pt(iEv, [0,-1])] for iEv in range(df.size() )]
mm_list = np.array(mm_list)
mm_list = np.array([mm for mm in mm_list if sum(mm**2)> 1e-5])
axes[0].hist(mm_list[:,0], bins =30)
axes[1].hist(mm_list[:,1]**2, bins = 30)
plt.suptitle('PT2')
plt.tight_layout()
plt.show()